Import library

ライブラリーをインポートする

In [ ]:
import pandas as pd
import time
import csv
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

This is the URL used to search for the toll fees. 

このURLで高速料金を探す

In [ ]:
options = Options()
options.add_argument('--headless')
url = "http://search.w-nexco.co.jp/route.php"

This will read the csv file of the list and delete the empty rows with no IC name available. It will also create a new list to input all the toll fees inside.

CSVファイルを読み込み、空セルを削除。リストを作る

In [ ]:
#read CSV
df = pd.read_csv('201807_ryokin.csv')
df_edit = df.dropna(subset=['入口','出口'])
full_kei = []

Run the whole program and iterate through each row. A time.sleep is put to add time for response. Prevent from being recognized as bot

プログラムを動かす。車種区分に車種によって変更してください。

In [ ]:
for index, row in df_edit.iterrows():
    
    #open the browser and navigate to the page
    #ブラウザ開いてURLに接続
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)

    #fill in the form
    #出発IC
    in_field = driver.find_element_by_name("fnm")
    in_field.send_keys(row['入口'])
    time.sleep(1)
    #到着IC
    out_field = driver.find_element_by_name("tnm")
    out_field.send_keys(row['出口'])
    time.sleep(1)
    
    #車種区分
    #Type of car: ("1"-軽・自動二輪, "2"-普通車, "3"-中型車, "4"-大型車, "5"- 特大車) 
    select_car_type = Select(driver.find_element_by_name("cartyp")).select_by_value("1")
    time.sleep(1)
    
    #日時
    #set date (2018/7/10)
    driver.find_element_by_class_name("calImgDiv_search").click()
    driver.find_element_by_id("day_2018_7_10_2_0").click
    #set time (午前10時)
    select_hour = Select(driver.find_element_by_id("sl_hour_id")).select_by_value("10")
    time.sleep(1)
    select_min = Select(driver.find_element_by_id("sl_min_id")).select_by_value("0")
    time.sleep(1)
    
    #通らない道路　（二つしかセットできない）
    #set detour (only 2 can be set)
    #"G1110"- C3 外環道, "G6000"-首都高速
    select_detour1 = Select(driver.find_element_by_id("detour1_id")).select_by_value("G1110")
    time.sleep(1)
    select_detour2 = Select(driver.find_element_by_id("detour2_id")).select_by_value("G6000")
    
    #検索する
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(3)
    driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
    time.sleep(2)
        
    #wait for page to load and click ETC料金順
    time.sleep(3)
    driver.find_element_by_id("pritab9").click()
    
    #wait for page to load and get the toll fee
    #ページを待つとデータを取る
    time.sleep(3)
    
    #通常(現金)
    normal_toll = driver.find_element_by_css_selector("span.toll-normal").get_attribute("innerText")
    #ETC
    etc_toll = driver.find_element_by_css_selector("span.toll-etc").get_attribute("innerText")
    #ETC2.0
    etc2_toll = driver.find_element_by_css_selector("span.toll-etc2").get_attribute("innerText")
    #休日
    kyujitsu_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[1]/dd/span').get_attribute("innerText")
    #深夜
    shinya_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div/div/dl[2]/dd/span').get_attribute("innerText")
    
    time.sleep(5)

    #close browser and session
    #ブラウザを閉じる
    driver.quit()
    
    #collected toll fess is put inside the list and made into list of list
    toll = [row['入口'],row['出口'],normal_toll, etc_toll, etc2_toll, shinya_toll, kyujitsu_toll]
    full_kei.append(toll)
    
    #wait time before next session
    time.sleep(10)

Change list into Pandas DataFrame.
Make another dataframe to exclude all unrelated symbols (円,分, etc.)

リストをPandasデータフレームに変更。もう一つのデータフレームを作り要らない記号を削除。

In [ ]:
pd_toll_kei = pd.DataFrame(full_kei)
all_kei = pd_toll_kei[pd_toll_kei.columns[2:]].replace('[\$,円,分]', '', regex=True).astype(int)

Export into Microsoft Excel file

エクセルにエクスポートする

In [ ]:
with pd.ExcelWriter('toll keisha.xlsx') as writer:
    pd_toll_kei.to_excel(writer, sheet_name='pd toll')
    all_kei.to_excel(writer, sheet_name='all kei')